In [69]:
## EU-DV-2 Threshold definition  – time-series
# The interface allow the expert user to define a threshold and select a 
# pollutant, a time window and a province and visualize a time series showing which days the 
# threshold has been exceeded and a percentage (exceeded days/total days).

# - button select a pollutant
# - button select threshold
# - button select start day
# - button select end day
# - button select start mounth
# - button select end mounth
# - button select start year
# - button select end year
# - button select a province 


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

selected_threshold = {"value": None}

# Output dinamici
output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()

# Dropdown inquinanti
dropdown = widgets.Dropdown(
    options=[
        "Ammoniaca", "Arsenico", "Benzo(a)pirene", "Benzene", "Biossido di Azoto",
        "Biossido di Zolfo", "BlackCarbon", "Cadmio", "Monossido di Azoto",
        "Monossido di Carbonio", "Nikel", "Ossidi di Azoto", "Ozono", "PM10",
        "PM10 (SM2005)", "Particelle sospese PM2.5", "Particolato Totale Sospeso",
        "Piombo"
    ],
    description='Inquinante:'
)

# Province della Lombardia
province_lombardia = [
    "Bergamo", "Brescia", "Como", "Cremona", "Lecco", "Lodi",
    "Mantova", "Milano", "Monza e della Brianza", "Pavia",
    "Sondrio", "Varese"
]

# Dropdown province (non visualizzato finché non vengono scelte le date)
dropdown_prov = widgets.Dropdown(
    options=province_lombardia,
    description='Provincia:'
)

# Funzione quando cambia dropdown inquinante: mostra slider
def on_dropdown_change1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output1:
            clear_output()
            slider = widgets.IntSlider(
                value=0,
                min=0,
                max=80,
                step=1,
                description=f'Soglia ({change["new"]}):',
                continuous_update=False
            )
            display(slider)

            # Funzione quando cambia lo slider: mostra i DatePicker
            def on_slider_change2(slider_change):
                if slider_change['type'] == 'change' and slider_change['name'] == 'value':
                    selected_threshold["value"] = slider_change["new"]
                    with output2:
                        clear_output()
                        global start_date, end_date  # Rende visibili fuori
                        start_date = widgets.DatePicker(description='Inizio:')
                        end_date = widgets.DatePicker(description='Fine:')

                        def check_dates(_):
                            if start_date.value and end_date.value:
                                with output3:
                                    clear_output()
                                    display(dropdown_prov)

                        # Monitora cambiamenti alle date
                        start_date.observe(check_dates, names='value')
                        end_date.observe(check_dates, names='value')

                        display(widgets.VBox([start_date, end_date]))

            slider.observe(on_slider_change2, names='value')

# Collega eventi
dropdown.observe(on_dropdown_change1, names='value')

# Visualizzazione finale
display(dropdown)
display(output1)
display(output2)
display(output3)




Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzo(a)pirene', 'Benzene', 'Biossido d…

Output()

Output()

Output()

ValueError: year 42024 is out of range

In [25]:
from datetime import date

# Assicurati che start_date.value ed end_date.value non siano None
if start_date.value and end_date.value:
    data = {
        "var_pollutant": dropdown.value,
        "var_start_date": start_date.value.strftime('%Y-%m-%d 00:00:00'),
        "var_end_date": end_date.value.strftime('%Y-%m-%d 24:00:00'),
        "var_province": dropdown_prov.value.upper(),
        "var_threshold": selected_threshold["value"]
    }

data



{'var_pollutant': 'Ozono',
 'var_start_date': '2024-12-20 00:00:00',
 'var_end_date': '2024-12-22 24:00:00',
 'var_province': 'MILANO',
 'var_threshold': 40}

In [26]:
# EU-DV-2 

import requests
import pandas as pd
import json

t=requests.post(url="http://127.0.0.1:5000/api/EU_DV_2", json=data) 

try:
    data = t.json() #This will convert the response to a json object
    print("The query returned", len(data), "rows")
    print(type(data))
except requests.exceptions.JSONDecodeError:
    print("Risposta non valida JSON!")
    print("Contenuto della risposta:", t.text)
    data = None

df = pd.DataFrame(data)
print(df)

df['data'] = pd.to_datetime(df['data'], errors='raise')
exceeded_days = df['data'].dt.date.nunique()
print(f"Giorni unici: {exceeded_days}")

The query returned 108 rows
<class 'list'>
                              data  valore
0    Fri, 20 Dec 2024 13:00:00 GMT    46.5
1    Fri, 20 Dec 2024 14:00:00 GMT    48.9
2    Fri, 20 Dec 2024 15:00:00 GMT    51.5
3    Fri, 20 Dec 2024 16:00:00 GMT    44.2
4    Fri, 20 Dec 2024 17:00:00 GMT    41.8
..                             ...     ...
103  Sat, 21 Dec 2024 12:00:00 GMT    42.9
104  Sat, 21 Dec 2024 13:00:00 GMT    52.4
105  Sat, 21 Dec 2024 14:00:00 GMT    51.2
106  Sat, 21 Dec 2024 15:00:00 GMT    51.0
107  Sat, 21 Dec 2024 16:00:00 GMT    50.2

[108 rows x 2 columns]
Giorni unici: 2


In [28]:
data = {"var_province": dropdown_prov.value.upper(),
        "var_pollutant": dropdown.value,
        "var_start_date": start_date.value.strftime('%Y-%m-%d 00:00:00'),
        "var_end_date": end_date.value.strftime('%Y-%m-%d 24:00:00'),
        } 

t=requests.post(url="http://127.0.0.1:5000/api/DV_7provinciamod", json=data) 

try:
    data = t.json() #This will convert the response to a json object
    print("The query returned", len(data), "rows")
    print(type(data))
except requests.exceptions.JSONDecodeError:
    print("Risposta non valida JSON!")
    print("Contenuto della risposta:", t.text)
    data = None

df = pd.DataFrame(data)
print(df)

df['data'] = pd.to_datetime(df['data'], errors='raise')
totaldays = df['data'].dt.date.nunique()
print(f"Giorni unici: {totaldays}")


The query returned 438 rows
<class 'list'>
                              data  valore
0    Fri, 20 Dec 2024 00:00:00 GMT     1.0
1    Fri, 20 Dec 2024 01:00:00 GMT     0.8
2    Fri, 20 Dec 2024 02:00:00 GMT     0.2
3    Fri, 20 Dec 2024 03:00:00 GMT     7.0
4    Fri, 20 Dec 2024 04:00:00 GMT    32.0
..                             ...     ...
433  Sun, 22 Dec 2024 20:00:00 GMT     4.8
434  Sun, 22 Dec 2024 21:00:00 GMT     2.8
435  Sun, 22 Dec 2024 22:00:00 GMT     2.6
436  Sun, 22 Dec 2024 23:00:00 GMT     2.8
437  Mon, 23 Dec 2024 00:00:00 GMT     4.7

[438 rows x 2 columns]
Giorni unici: 4


In [23]:
# visualize a time series showing which days the threshold has been exceeded and a percentage (exceeded days/total days).

percentage=0
percentage=(exceeded_days/totaldays)*100
print(f"Percentage of days in which the threshold has been exceeded : {percentage}")


Percentage of days in which the threshold has been exceeded : 50.0
